### Parsing 

- Inclui todos os foros, matérias, unidades e feitos em andamento dos documentos

In [ ]:
import os
import re
import gc
import time
import numpy as np
import pandas as pd
import logging
from PyPDF2 import PdfReader
from concurrent.futures import ThreadPoolExecutor, as_completed
from natsort import natsorted
from tqdm import tqdm
import multiprocessing

# --- Configuração de logging ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler()]
)

# --- Padrões encontrados nos PDFs ---
MATERIAS = [
    "CÍVEL", "CRIMINAL", "EXECUÇÃO FISCAL", "INFÂNCIA",
    "JUIZADO CRIMINAL", "JUIZADO ESPECIAL",
    "JUIZADO FAZENDA PÚBLICA", "SETOR FAZENDA PÚBLICA"
]

PADRAO_MATERIA = re.compile(
    r"Foro:\s*(.+?)\s+Unidade:\s*(.+?)\s+Matéria:\s*"
    r"(CÍVEL|CRIMINAL|EXECUÇÃO FISCAL|INFÂNCIA|JUIZADO CRIMINAL|"
    r"JUIZADO ESPECIAL|JUIZADO FAZENDA PÚBLICA|SETOR FAZENDA PÚBLICA)",
    flags=re.IGNORECASE
)

PADRAO_FEITOS = re.compile(r"(?i)\bTotal\s+de\s+Feitos\s+em\s+Andamento\s*(\d[\d\.]*)")

# --- Extração de texto do PDF ---
def extrair_texto_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        texto = " ".join(
            t.replace('\n', ' ').strip()
            for page in reader.pages
            for t in [page.extract_text()]
            if t
        )
        return re.sub(r"Considerações\s+para\s+análise\s+dos\s+dados.*$", "", texto, flags=re.IGNORECASE)
    except Exception as e:
        logging.error(f"Erro ao extrair texto de {file_path}: {e}")
        return ""

# --- Extração estruturada por bloco ---
def extrair_blocos_completo(texto):
    matches = list(PADRAO_MATERIA.finditer(texto))
    blocos = []
    for i, m in enumerate(matches):
        start_idx = m.start()
        end_idx = matches[i + 1].start() if i + 1 < len(matches) else len(texto)
        bloco = texto[start_idx:end_idx]

        feitos = PADRAO_FEITOS.search(bloco)
        feitos_tot = int(feitos.group(1).replace(".", "")) if feitos else np.nan

        blocos.append({
            "Foro": m.group(1).strip(),
            "Unidade": m.group(2).strip(),
            "Matéria": m.group(3).strip(),
            "regex": bloco.strip(),
            "feitos_tot": feitos_tot
        })

    return blocos

# --- Processamento individual ---
def processar_pdf(nome_arquivo, caminho_pdf, safra):
    texto = extrair_texto_pdf(caminho_pdf)
    if not texto.strip():
        logging.warning(f"Texto vazio para {nome_arquivo}")
        return []

    blocos = extrair_blocos_completo(texto)
    if not blocos:
        logging.warning(f"Nenhum bloco identificado em {nome_arquivo}")

    for bloco in blocos:
        bloco["Safra"] = safra
        bloco["Arquivo"] = nome_arquivo
    return blocos

# --- Processamento em lote ---
def processar_pdfs_em_lote(pasta_base, max_threads=None):
    if not os.path.exists(pasta_base):
        raise FileNotFoundError(f"Pasta '{pasta_base}' não encontrada.")

    if max_threads is None:
        max_threads = max(1, multiprocessing.cpu_count() - 1)

    safras = [s for s in os.listdir(pasta_base) if os.path.isdir(os.path.join(pasta_base, s))]
    todos_dados = []
    inicio_total = time.time()

    for safra in safras:
        logging.info(f"Processando safra: {safra}")
        inicio_safra = time.time()
        pasta_safra = os.path.join(pasta_base, safra)
        arquivos_pdf = natsorted([f for f in os.listdir(pasta_safra) if f.endswith(".pdf")])
        resultados = []

        with ThreadPoolExecutor(max_workers=max_threads) as executor:
            futuros = [
                executor.submit(processar_pdf, nome, os.path.join(pasta_safra, nome), safra)
                for nome in arquivos_pdf
            ]

            for futuro in tqdm(as_completed(futuros), total=len(futuros), desc=f"Processando {safra}"):
                resultado = futuro.result()
                if resultado:
                    resultados.extend(resultado)

        df_safra = pd.DataFrame(resultados)
        df_safra["feitos_tot"] = pd.to_numeric(df_safra["feitos_tot"], errors="coerce").astype("Int64")
        todos_dados.append(df_safra)

        del resultados, df_safra
        gc.collect()

        logging.info(f"Safra {safra} finalizada: {len(arquivos_pdf)} arquivos em {time.time() - inicio_safra:.1f}s")

    if todos_dados:
        df_final = pd.concat(todos_dados, ignore_index=True)
        colunas = ["Foro", "Unidade", "Matéria", "Arquivo", "regex", "Safra", "feitos_tot"]
        df_final = df_final[colunas]

        caminho_csv = os.path.join(pasta_base, "foros_unidades_feitos_geral.csv")
        df_final.to_csv(caminho_csv, index=False, encoding="utf-8-sig")
        logging.info(f"CSV salvo com {len(df_final)} registros: {caminho_csv}")
        logging.info(f"Tempo total de execução: {time.time() - inicio_total:.1f} segundos")
    else:
        logging.warning("Nenhum dado foi consolidado.")


### Execução


In [ ]:
processar_pdfs_em_lote("data-raw/")